In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
import numpy as np

### First Attempt just using Beautiful Soup

In [ ]:
levels = ['college', 'high-school']
sports = ['football', 'basketball', 'baseball', 'womens-basketball', 'volleyball', 'gymnastics',
         'mens-lacrosse', 'womens-lacrosse', 'mens-soccer', 'womens-soccer', 'softball', 'womens-track',
         'mens-golf', 'womens-golf', 'mens-hockey', 'womens-hockey', 'mens-swimming', 'womens-swimming']

all_names = []

for level in levels:
    for sport in sports:

        URL = f'https://www.on3.com/nil/rankings/player/{level}/{sport}/'

        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        results = soup.find(class_="NilRankingsPageComponent_nilRankingsList__t14Ao")

        try:
            names_sport = results.find_all('a', class_="MuiTypography-root MuiLink-root MuiLink-underlineNone MuiTypography-h5 MuiTypography-colorPrimary")
            for name in names_sport:
                all_names.append(name)
        except:
            # print("Whoops! No athletes appear to have NIL deal for {} {}".format(level, sport))
            continue
    #end
#end

In [7]:
print(len(all_names))

414


It looks like there were only 414 athletes using this section of the On3 database... This is such a small number of athletes that we should investiage looking at different sections of the website

### Using Selenium & Beatuiful Soup

In [2]:
sports = ['basketball']
#years = ['2022','2023','2024','2025','2026']
years = ['2023']

exp = []
athlete_grade = []
pos_height_weight = []
ages = []
ranks = []
high_school = []
home_town = []
colleges = []
college_status = []
college_distance = []
num_offers = []
NIL_val = []
instagram_followers = []
twitter_followers = []
tiktok_followers = []

for year in years:
    for sport in sports:
        
        # Selenium Driver to click on "Load More"
        driver = webdriver.Chrome()
        URL = f'https://www.on3.com/db/rankings/industry-player/{sport}/{year}/'
        driver.get(URL)
        
        ###########################################################################################################################
        # Click the "Load More" button such that all athelete links are visible
        # while True:
        #     try:
        #         load_more_button = WebDriverWait(driver, 10).until(
        #             EC.element_to_be_clickable((By.XPATH, "//span[@class='MuiButton-label' and contains(text(), 'Load More')]"))
        #         )
        #         load_more_button.click()
        #         time.sleep(10)
        #     except:
        #         break

        ###########################################################################################################################
        
        ###########################################################################################################################
        # Get all athlete names & associated links
        # New Information: names, links
        page_source = driver.page_source
        page_soup = BeautifulSoup(page_source, 'html.parser')
        results = page_soup.find(class_="PlayerRankings_playerRankings__jvfFg")

        athletes = results.find_all('a', class_="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-h5 MuiTypography-colorPrimary")
        names = [athlete.text for athlete in athletes]

        # Generate a list of links that we can iterate through
        links = [athlete['href'] for athlete in athletes]
        base_url = "https://www.on3.com/"
        athlete_links = [urljoin(base_url, link) for link in links]

        ###########################################################################################################################

        for i, athlete_link in enumerate(athlete_links):

            ###########################################################################################################################
            # Information from the "Player" page for each of the athletes
            # New Information: exp, athlete_grade, ages, ranks, high_school, home_town, pos_height_weight
            try:
                driver.get(athlete_link)
                time.sleep(3)

                athlete_source = driver.page_source
                page_soup = BeautifulSoup(athlete_source, 'html.parser')

                CollegeRankingInfo = page_soup.find(class_='CollegeRanking_info__LM3nn')

                if CollegeRankingInfo:
                    exp_year = CollegeRankingInfo.find_all(class_='MuiTypography-root CollegeRanking_span__qtAfW MuiTypography-subtitle1 MuiTypography-colorTextPrimary')
                    if len(exp_year) == 2:
                        exp.append(exp_year[0].text)
                        athlete_grade.append(exp_year[1].text)
                    else:
                        exp.append(np.nan)
                        athlete_grade.append(np.nan)
                    age = CollegeRankingInfo.find(class_='MuiTypography-root CollegeRanking_span__qtAfW MuiTypography-subtitle1 MuiTypography-colorPrimary')
                    ages.append(age.text)
                else:
                    exp.append(np.nan)
                    athlete_grade.append(np.nan)
                    ages.append(np.nan)
                
                RecruitModuleInfo = page_soup.find(class_='RecruitModule_info__Ugxqd')

                if RecruitModuleInfo:
                    ranking = RecruitModuleInfo.find(class_='RecruitModule_rating__sONqb')
                    ranks.append(ranking.text)

                    homeInfo = RecruitModuleInfo.find_all(class_='MuiTypography-root RecruitModule_span__KmmzN MuiTypography-subtitle1 MuiTypography-colorTextPrimary')
                    high_school.append(homeInfo[-2].text)
                    home_town.append(homeInfo[-1].text)
                else:
                    ranks.append(np.nan)
                    high_school.append(np.nan)
                    home_town.append(np.nan)

                Attributes = page_soup.find(class_='MeasurementInfo_info__IHmGD')

                if Attributes:
                    dummy = Attributes.find_all(class_='MuiTypography-root MeasurementInfo_text__dCryI MuiTypography-body1 MuiTypography-colorTextPrimary')
                    if len(dummy) == 2:
                        pos_height_weight.append(dummy[1].text)
                    else:
                        pos_height_weight.append(np.nan)
            except:
                print("Error on the 'Player' page for athlete: {}".format(names[i]))
                print("Removing all previous information about this player! (i.e name)")
                names.pop(i)
                continue
            ###########################################################################################################################
            
            ###########################################################################################################################
            # Information from the "Recruiting" page for each of the athletes
            # New Information: colleges (the college this athlete is targeting), college_status, college_distance, num_offers
            driver.get(urljoin(athlete_link,'recruiting/'))
            time.sleep(3)
            athlete_source = driver.page_source
            page_soup = BeautifulSoup(athlete_source, 'html.parser')
            
            # Find url for all teams and navigate to page
            all_team_link = page_soup.find(class_='MuiTypography-root MuiLink-root MuiLink-underlineHover PlayerInterestsModule_text__kjqNU MuiTypography-caption MuiTypography-colorPrimary')
            driver.get(urljoin(base_url,all_team_link['href']))
            time.sleep(3)
            athlete_source = driver.page_source
            page_soup = BeautifulSoup(athlete_source, 'html.parser')
            
            try:
                RecruitColleges = page_soup.find_all(class_='PlayerInterestsItem_teamContainer__vjQkf')
                if RecruitColleges:
                    dummy_colleges=[]
                    count = 0
                
                    for college in RecruitColleges:
                        college_name = college.find(class_='MuiTypography-root MuiLink-root MuiLink-underlineNone PlayerInterestsItem_teamName__FeBHv MuiTypography-h5 MuiTypography-colorPrimary')
                        college_statuses = college.find(class_='MuiTypography-root PlayerInterestsItem_status__1_rA8 MuiTypography-subtitle1 MuiTypography-colorTextPrimary')
                        college_dist = college.find(class_='MuiTypography-root PlayerInterestsItem_distanceText__KJhj3 MuiTypography-caption MuiTypography-colorTextPrimary')
                        if count == 0:
                            colleges.append(college_name.text)
                            college_status.append(college_statuses.text)
                            college_distance.append(college_dist.text)
                        count +=1
                    num_offers.append(count)
                else:
                    colleges.append(np.nan)
                    college_status.append(np.nan)
                    college_distance.append(np.nan)
                    num_offers.append(np.nan)
                    
            except:
                print("Error on the 'Recruiting' page for athlete: {}".format(names[i]))
                print("Removing all previous information about this player! (i.e name, exp, athlete_grade, ages, ranks, high_school, home_town, pos_height_weight)")
                names.pop(i)
                exp.pop(i)
                athlete_grade.pop(i)
                ages.pop(i)
                ranks.pop(i)
                high_school.pop(i)
                home_town.pop(i)
                pos_height_weight.pop(i)
                continue
            ###########################################################################################################################
            
            ###########################################################################################################################
            # Information from the "NIL" page for each of the athletes
            # New Information: NIL_val, instagram_followers, twitter_followers, tiktok_followers
            driver.get(urljoin(athlete_link,'nil/'))
            time.sleep(3)
            athlete_source = driver.page_source
            page_soup = BeautifulSoup(athlete_source, 'html.parser')
            
            try:
                RecruitNIL = page_soup.find(class_='NilValuationCircle_nilCircleValue__wzomB')

                if RecruitNIL:
                    NIL_val.append(RecruitNIL.text)
                else:
                    NIL_val.append(np.nan)
                
                RecruitSocials = page_soup.find(class_="NilSocialValuations_socialValuations__MeR7O")

                instagram = np.nan
                twitter = np.nan
                tiktok = np.nan

                if RecruitSocials:
                    socials = RecruitSocials.find_all(class_="NilSocialValuations_platform__3qBMy")
                    for social in socials:
                        social_name = social.find('a')['href']
                        if 'instagram' in social_name:
                            instagram = social.find(class_='MuiTypography-root NilSocialValuations_platformFollowers__sW8kK MuiTypography-body1 MuiTypography-colorTextPrimary').text
                        elif 'twitter' in social_name:
                            twitter = social.find(class_='MuiTypography-root NilSocialValuations_platformFollowers__sW8kK MuiTypography-body1 MuiTypography-colorTextPrimary').text
                        elif 'tiktok' in social_name:
                            tiktok = social.find(class_='MuiTypography-root NilSocialValuations_platformFollowers__sW8kK MuiTypography-body1 MuiTypography-colorTextPrimary').text
                
                instagram_followers.append(instagram)
                twitter_followers.append(twitter)
                tiktok_followers.append(tiktok)
                
            except:
                print("Error on the 'NIL' page for athlete: {}".format(names[i]))
                print("Removing all previous information about this player! (i.e name, exp, athlete_grade, ages, ranks, high_school, home_town, pos_height_weight, colleges, college_status, college_distance, num_offers)")
                names.pop(i)
                exp.pop(i)
                athlete_grade.pop(i)
                ages.pop(i)
                ranks.pop(i)
                high_school.pop(i)
                home_town.pop(i)
                pos_height_weight.pop(i)
                colleges.pop(i)
                college_status.pop(i)
                college_distance.pop(i)
                num_offers.pop(i)
                continue
            ###########################################################################################################################
                
        #end
    #end
#end

# Close the instance of the webpage
driver.quit()

In [3]:
# print(instagram_followers)
# print(twitter_followers)
# print(tiktok_followers)

In [6]:
print(len(names), len(exp), len(pos_height_weight), len(athlete_grade), len(ages), len(ranks), len(high_school), len(home_town),
      len(colleges), len(college_status), len(college_distance), len(num_offers), len(NIL_val),
      len(instagram_followers), len(twitter_followers),len(tiktok_followers))
print(names[0], exp[0], pos_height_weight[0], athlete_grade[0], ages[0], ranks[0], high_school[0])
print(home_town[0], colleges[0], college_status[0], college_distance[0], num_offers[0])
print(NIL_val[0], instagram_followers[0], twitter_followers[0], tiktok_followers[0])

50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50
Isaiah Collier 2023 - present PG / 6-3 / 190 Freshman 18 99.39 Wheeler
Marietta, GA USC Enrolled 1921 mi. 17
$594K 85K 2.8K nan
